In [4]:
import lmfit
from lmfit import minimize, Parameters, fit_report
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import Analysis
from scipy.optimize import least_squares
%matplotlib

Using matplotlib backend: TkAgg


In [241]:
def sigmoid(x,v0):
    return 1.0 / (1 + np.exp(np.array((-x+v0)/0.1,dtype='float')))


def residual(params, x, y, v0, eps_data):

    model_r = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
    model_l = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
    
    out_r = (y-model_r)*sigmoid(x,v0)
    out_l = (y-model_l)*(1.0-sigmoid(x,v0))
    
    return (out_r+out_l)/eps_data



In [5]:
def search(seq, t):  
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max <= min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [243]:
def regression(x_min, x_max, data, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
    
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)
    
    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]
    
    
    
    a = (np.percentile(y,99)-np.percentile(y,1))/2.0
    
    params = []
    bounds = ([],[])
    
    h = np.mean(y)
    params.append(h) #height
    bounds[0].append(h*0.99)
    bounds[1].append(h*1.01)
    
    params.append(a) #amplitude
    bounds[0].append(a*0.8)
    bounds[1].append(a*1.2)
    
    params.append(0.63) #frequency
    bounds[0].append(0.6)
    bounds[1].append(0.77)
    
    params.append(0.0) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    params.append(np.random.uniform(0.1,1.9)) #delta phase
    bounds[0].append(0.0)
    bounds[1].append(2)
    
    params.append(0.0) #slope
    bounds[0].append(-0.02)
    bounds[1].append(0.02)
    
    v0 = (x_min+x_max)/2.0
    
    out = least_squares(residual, x0 = params, bounds=bounds, loss=loss, x_scale = x_scale, args=(x , y, v0, eps))
    
    if plot:
        params = out['x']
        model_r = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
        model_l = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    
    return out    

In [339]:
def multi_reg_param_ci(data, x_min, x_max, result, eps): 
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1

    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)

    N = float(i_max - i_min)
    k = len(result.x)

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    res = result.fun*eps

    sigma_r = 1/(N-k)*np.sum(res**2)
    
    inv_j = np.linalg.inv(np.dot(result.jac.transpose(),result.jac))

    cov = sigma_r*inv_j
    
    return np.sqrt(np.diag(cov))

In [244]:
def multi_regression(x_min, x_max, data, n = 20, loss = 'linear', eps = 0.1, x_scale = 1.0, plot=False):
    cost = np.inf
    result_hold = None
    for _ in range(n):
        result = regression(x_min, x_max, data, loss = loss, eps = eps)
        if result.success:
            
            
            
            
            
            
            
            
            
            
            if result.cost < cost:
                cost = result.cost
                result_hold = result
                
    if plot:
        plt.figure()
        
        i_min = search(data.x,x_min)
        i_max = search(data.x,x_max)+1
    
        x = data.x[i_min:i_max]
        y = data.y[i_min:i_max]
        
        v0 = (x_min+x_max)/2.0
        
        params = result_hold.x
        model_r = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
        model_l = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    return result_hold

In [335]:
def sweep_regression(data, d = 6.0, c = 20, n = 20, loss = 'linear', eps = 0.1, x_scale = 1.0, plot=False):
    out = list()
    x_max = max(data.x)
    x_min = min(data.x)
    step = (x_max-x_min-2*d)/c
    for i in range(c):
        center = x_min + d +  i*step
        low = center - d
        high = center + d
        result = multi_regression(low, high, data, n, loss = loss, eps = eps, x_scale = x_scale, plot = plot)
        
        res_dict = {'result':result, 'x_min':low, 'x_max':high, 'eps':eps}
        r_2 = r_squared(data,res_dict)
        res_dict['r_2'] = r_2
        
        out.append(res_dict)
    return out
    

In [103]:
def run_regression(data, d = 6.0, c = 20, n = 20, loss = 'linear', eps = 0.01, x_scale = 1.0, plot=False):
    count = 0
    out = []
    for i in data:
        print('Running sweep ' + str(count))
        out.append(sweep_regression(i, d = d, c = c, n=n, eps = eps, x_scale = x_scale, loss = loss, plot = plot))
        count += 1
    return out

In [337]:
def param_ci(data,result):
    x_min = result['x_min']
    x_max = result['x_max']
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1

    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)

    N = float(i_max - i_min)
    k = len(result['result'].x)

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    res = result['result'].fun*result['eps']

    sigma_r = 1/(N-k)*np.sum(res**2)
    
    inv_j = np.linalg.inv(np.dot(result['result'].jac.transpose(),result['result'].jac))

    cov = sigma_r*inv_j
    
    return np.sqrt(np.diag(cov))
    


In [245]:
def plot(data,result):
    x_min = result['x_min']
    x_max = result['x_max']
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    params = result['result'].x
    model_r = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
    model_l = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
    plt.plot(x,y)
    plt.plot(x,model_r,c='r')
    plt.plot(x,model_l,c='k')

In [336]:
def r_squared(data,result):
    x_min = result['x_min']
    x_max = result['x_max']
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0
    
    res = result['result'].fun*result['eps']
    SSres = np.sum(res*res)
    
    ybar = np.mean(y)
    
    SStot = np.sum((y-ybar)**2)

    return 1 - SSres/SStot

In [11]:
run = Analysis.generate_2014()

In [24]:
test = regression(-19.5464, -8.39718,run.prune_data[0],eps = 1.0, plot = True)

In [64]:
test_multi = multi_regression(-19.5464, -8.39718, run.prune_data[0], eps = 0.01, plot = True, x_scale = [10.0,0.1,0.1,1.0,1.0,1.0], loss = 'huber')

In [117]:
test_sweep_reg = sweep_regression(run.prune_data[0], d = 4.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,1.0], loss = 'huber')

In [246]:
test_run_reg = run_regression(run.prune_data, d = 4.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,1.0], loss = 'huber')

Running sweep 0
Running sweep 1
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6
Running sweep 7
Running sweep 8
Running sweep 9
Running sweep 10
Running sweep 11
Running sweep 12
Running sweep 13
Running sweep 14


In [247]:
filenum = 0
t = test_run_reg[filenum]
count = 0
for i in t:
    if i['r_2'] > 0.80 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['r_2']) + ', '+ str(i['result'].x[4]))
    count += 1

In [236]:
test_sweep_reg = sweep_regression(run.prune_data[filenum], d = 2.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,1.0], loss = 'huber')

In [237]:
count = 0
for i in test_sweep_reg:
    if i['r_2'] > 0.80 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['r_2']) + ', '+ str(i['result'].x[4]))
    count += 1

In [150]:
for i in test_sweep_reg:

    plt.figure()
    plot(run.prune_data[filenum],i)

In [215]:
for i in test_sweep_reg[45:]:
    print(i['r_2'])
    plt.figure()
    plot(run.prune_data[filenum],i)
    

0.647814567513
0.422638938379
0.76976327421
0.759036992857
0.797299671867


In [228]:
filenum = 4
t = test_run_reg[filenum]
count = 0
for i in t:
    if i['r_2'] > 0.90 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['r_2']) + ', '+ str(i['result'].x[4]))
    count += 1

In [235]:
for i in [t[12],t[34]]:
    print(i['r_2'])
    plt.figure()
    plot(run.prune_data[filenum],i)
    

0.928063299689
0.823118878632


In [230]:
plot(run.prune_data[filenum],t[34])

In [300]:
t[30]

{'r_2': 0.76307924499903679,
 'result':  active_mask: array([-1,  0,  1,  0,  0,  0])
        cost: 12233.957680301193
         fun: array([-11.75794639, -11.07212278, -12.70640812, ...,  -3.05563954,
        -3.21033859,  -3.13840104])
        grad: array([  3.46181664e+04,  -1.55140544e-04,  -1.82060641e+04,
         2.28175949e-04,   5.86811776e-04,  -8.40815619e-04])
         jac: array([[ -1.49011612e-06,  -8.85021834e-08,   7.20922468e-06,
         -3.57261787e-07,   0.00000000e+00,   6.70622704e-06],
       [ -1.49011612e-06,  -9.05522768e-08,   7.20798727e-06,
         -3.57232132e-07,   0.00000000e+00,   6.70355966e-06],
       [ -1.49011612e-06,  -9.22701915e-08,   7.20693976e-06,
         -3.57206755e-07,   0.00000000e+00,   6.70132447e-06],
       ..., 
       [ -1.49011612e-06,  -7.69185604e-08,  -3.99636626e-06,
          3.57416450e-07,   1.12285683e-06,  -6.70161420e-06],
       [ -1.49011612e-06,  -7.53029639e-08,  -3.99608417e-06,
          3.57436259e-07,   1.1229191

In [281]:
inv_j = np.linalg.inv(np.dot(t[32]['result'].jac.transpose(),t[32]['result'].jac))

In [267]:
t[32]['result'].x.reshape(-1,1)

array([[  1.14659368e+01],
       [  2.17644378e-01],
       [  7.25759100e-01],
       [  2.61975696e+00],
       [  8.99411725e-01],
       [ -6.36545487e-03]])

In [323]:
hold = list()
for i in t:
    hold.append(param_ci(run.prune_data[filenum],i))
    
hold = np.array(hold)
plt.plot(hold[:,4])

In [331]:
plot(run.prune_data[0],t[32])

In [328]:
data = run.prune_data[filenum]
result = t[32]

x_min = result['x_min']
x_max = result['x_max']
i_min = search(data.x,x_min)
i_max = search(data.x,x_max)+1

x = data.x[i_min:i_max]
y = data.y[i_min:i_max]

In [329]:
plt.plot(x,y-result['result'].fun)

In [327]:
plt.figure()

In [334]:
plt.plot(x,result['result'].fun*0.01)